In [1]:
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
import numpy as np
from scipy import optimize, arange
from scipy.integrate import quad,dblquad

# Model

As explained in the note, we consider both a one and a two period model.

To keep things simple, assume that the bivariate distribution $f(y,u) = \frac{1}{\bar y \alpha \bar y}$. That is, $y$ is uniformly distributed on interval $[0,\bar y]$ and $u$ is uniformly distributed on $[0,\bar u]$ with $\bar u = \alpha \bar y$. Further, $u$ and $y$ are independent.

The probability that someone needs treatment is given by $\mu \in [0,1]$.


In [2]:
mu = 0.4
bar_y = 2000.0
alpha = 1.5
bar_u = alpha*bar_y
y_array = arange(0,bar_y,1)
u_array = arange(0,alpha*bar_y,1)

def f(y,u):
    return 1.0/(bar_y**2*alpha)

An agent with health care expenditure $y$ and deductible $D$ pay herself the amount given by the function `cost`.

In [3]:
def cost(y,D):
    if y < D:
        cost = y
    else: cost = D
    return cost


For the figures here, we work with two deductible levels:

In [4]:
deductible_1 = 200
deductible_2 = 400

## one period model

In the one period model, the agent gets one draw $y,u$ and decides whether or not to accept this treatment and pay `cost(y,D)` for it. The function `critical_u_one_period` defines a benchmark value such that treatments with $u$ exceeding `critical_u_one_period(y,D)` are accepted by the agent. In the one period model, this benchmark value is given by the agent's expenditure `cost(y,D)`.



In [5]:
def critical_u_one_period(y,D):
    critical_u = cost(y,D)
    return critical_u

def density(y,D):
    return mu*(1-cost(y,D))/bar_u

density_1 = [round(density(y,deductible_1),2) for y in y_array]
density_2 = [round(density(y,deductible_2),2) for y in y_array]

The number of people consuming no health care $G(0)$ is defined by the function `G_0(D)`. It is given by the people who do not fall ill ($1-\mu$) in the first place. Second, agents who do fall ill and get treatments $y,u$ offered with $u<$`cost(y,D)` do not consume health care either.

The density $g(z) = \mu \int_{cost(z,D)}^{+\infty} f(z,u)du$ equals for uniform $f(y,u)$ the expression $g(z) = \mu \frac{\bar u - cost(z,D)}{\bar u \bar y}$.

`g_normalized` gives $g(z)/(1-G(0))$. The Python function `quad` integrates the function $g(y,D)$ over $y$ from 0 to $\bar y$.

In [6]:

def G_0(D):
    return 1-mu+mu*(0.5*D**2+D*(bar_y-D))/(bar_y*bar_u)

def g(y,D):
    return mu*(bar_u-cost(y,D)/(bar_u*bar_y))

def g_normalized(y,D):
    return g(y,D)/quad(lambda x: g(x,D), 0, bar_y)[0]

In [9]:
traceg1= Scatter(
      x = y_array,
      y = [g_normalized(y,deductible_1) for y in y_array],
      mode='markers+lines', 
      name = 'one period, deductible = 200',
      #text = y
      )
traceg2= Scatter(
      x = y_array,
      y = [g_normalized(y,deductible_2) for y in y_array],
      mode='markers+lines', 
      name = 'one period, deductible = 400',
      #text = y
      )

In [15]:
layout = Layout(
    title='Normalized density health care use',
    xaxis=XAxis(
        title='$y$',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        # range=[0,2.0*deductible_2]
    ),
    yaxis=YAxis(
        title='$g/(1-G(0))$',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        showticklabels=False
        # range=[0,1.2]
    )
)

data = Data([traceg1, traceg2])
fig = Figure(data=data, layout=layout)
py.plot(fig, filename='density expenditure')
tls.embed("https://plot.ly/~janboone/158")

Hence we see that with the higher deductible (400 vs 200) that the normalized density is higher for low $y$ and lower for high $y$. For high $y$ this corresponds to a fall in $g(y)$ itself. For low $y$, $g(y)$ does not change with $D$ but higher $D$ leads to higher $G(0)$. Hence the normalized density at low $y$ is higher for $D=400$ than for $D=200$.

The following cell generates the critical $u$ level in $y,u$ space: for $u$ values above (below) this line the treatment is (not) accepted by the agent.

In [16]:
trace1= Scatter(
      x = y_array,
      y = [critical_u_one_period(y,deductible_1) for y in y_array],
      mode='markers+lines', 
      name = 'one period, deductible = 200',
      text = density_1
      )
trace2= Scatter(
      x = y_array,
      y = [critical_u_one_period(y,deductible_2) for y in y_array],
      mode='markers+lines', 
      name = 'one period, deductible = 400',
      text = density_2
      )

## two period model

In the two period model, we focus on consumption in the first period. The main effect that we are after is that the density $g(y)$ for low $y$ is affected. In particular, a treatment offer $y_1,u_1$ is accepted in the first period if $u_1 \geq c(y_1,D) - \mu(v(D - y_1)-v(D))$. As the figure below shows: a change in $D$ affects this critical value for $u$ even at low values of $y$.

We still need to characterize the second period effect, but as there is an effect alreay in the first period, one expects that there will be an effect as well when both periods are taken into account.

In [17]:

def integrand(y,u,D):
    return (u-cost(y,D))/(bar_y*bar_u)

def v(D):
    v = dblquad(lambda x, y: integrand(x,y,D), 0, bar_y, lambda x: cost(x,D), lambda x: bar_u)
    return v[0]

def critical_u_two_periods(y,D):
    if y < D:
        critical_u = cost(y,D)-mu*(v(D-y)-v(D))
    else: critical_u = cost(y,D)-mu*(v(0)-v(D))
    return critical_u

In [18]:
trace3= Scatter(
      x = y_array,
      y = [critical_u_two_periods(y,deductible_1) for y in y_array],
      mode='markers+lines', 
      name = 'two periods, deductible = 200',
      text = 'first period'
      )
trace4= Scatter(
      x = y_array,
      y = [critical_u_two_periods(y,deductible_2) for y in y_array],
      mode='markers+lines', 
      name = 'two periods, deductible = 400',
      text = 'first period'
      )

In [19]:
layout = Layout(
    title='First period health care use',
    xaxis=XAxis(
        title='$y$',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        range=[0,2.0*deductible_2]
    ),
    yaxis=YAxis(
        title='$u$',
        titlefont=Font(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        #range=[0,deductible_2+100]
    )
)

data = Data([trace1, trace2,trace3, trace4])
fig = Figure(data=data, layout=layout)
py.plot(fig, filename='deductibles')
tls.embed("https://plot.ly/~janboone/143")

## things to note in this graph

First by clicking in the lines in the "legenda" you can make them appear and disappear. In this way, you can easily compare one-period-D200 with one-period-D400, but also one-period-D200 with two-periods-D200 etc.

Clearly, the critical $u$ (above which a treatment is accepted) is lower in the two period model. In the latter, consuming in the first period has as an additional benefit a lower (remaining) deductible in the second period. This makes the agent more eager to accept treatment (compared to the one period model).

In both models, a higher deductible raises the critical $u$: health care consumption falls with the deductible. In the one period model, this only happens for $y > 200$. As noted above, there is no effect of $D$ for $y$ below the lowest deductible ($D=200$ in this case).

This is different in the two period model. Even for $y < 200$ the two lines diverge (you may want to click away the one period lines to see this better).